<a href="https://colab.research.google.com/github/kruttikajain/ACL2019-Reducing-Gender-Bias-in-Word-Level-Language-Models-Using-A-Gender-Equalizing-Loss-Function/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/Colab Notebooks/MSProject

/content/gdrive/My Drive/Colab Notebooks/MSProject


In [3]:
pip install import_ipynb

In [4]:
pip install unidecode

In [5]:
import import_ipynb
import model
#import trial

In [6]:
import preprocess
import data_v3

In [7]:
 #import datav3
''' from google.colab import files

uploaded = files.upload()'''

' from google.colab import files\n\nuploaded = files.upload()'

In [8]:
 #import preprocess
 '''from google.colab import files

uploadedd = files.upload()'''

'from google.colab import files\n\nuploadedd = files.upload()'

In [9]:
#args = parser.parse_args(args=[])

In [10]:
!nvidia-smi

Thu Apr 22 07:22:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
#### (1) Check if PyTorch and CUDA are available
import torch
torch.cuda.is_available()
#### (2) Inspect GPU
!nvidia-smi
#### (3) Test PyTorch
import torch
print(torch.ones(3,2))

Thu Apr 22 07:22:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
!bash pytorch041_cuda92_colab.sh

bash: pytorch041_cuda92_colab.sh: No such file or directory


In [13]:
%load_ext nvcc_plugin

directory /content/gdrive/My Drive/Colab Notebooks/MSProject/src already exists
Out bin /content/gdrive/My Drive/Colab Notebooks/MSProject/result.out


In [14]:
# coding: utf-8
import argparse
import time
import math
import torch
import torch.nn as nn
from torch.autograd import Variable

import data_v3
import model
import preprocess
#import jams
import os
import pandas as pd
from sklearn.model_selection import ShuffleSplit
import json

import numpy as np
import random
# seed = random.seed(20180330)
#python -u  model/training_loss.py --dropout 0.25 --tied --lr 20 --anneal 4 --cuda --lamda 0 --glove --batch_size 48 --data ./data/preprocessed_50000/data --vocab ./data/preprocessed_50000/VOCAB.txt --save ./savedmodel/model_glove.pt --epochs 100 --glove_path ./glove/glove.txt
#λ0.01 0.492 0.245 1.445 118.585 0.111 9.306 0.077
'''We use a pre-trained 300-dimensional word embedding, GloVe, by Pennington et al. (2014). We
apply random search to the hyperparameter tuning
of the LSTM language model. The best hyperparameters are as follows: 2 hidden layers each with
300 units, a sequence length of 35, a learning rate
of 20 with an annealing schedule of decay starting from 0.25 to 0.95, a dropout rate of 0.25 and
a gradient clip of 0.25. We train our models for
150 epochs, use a batch size of 48, and set early
stopping with a patience of 5'''
parser = argparse.ArgumentParser(description='PyTorch Wikitext-2 RNN/LSTM Language Model')
parser.add_argument('--data', type=str, default='/content/gdrive/My Drive/Colab Notebooks/MSProject/Output/data',
                    help='location of the data corpus')
parser.add_argument('--model', type=str, default='LSTM',
                    help='type of recurrent net (RNN_TANH, RNN_RELU, LSTM, GRU)')
parser.add_argument('--emsize', type=int, default=300,
                    help='size of word embeddings')
parser.add_argument('--nhid', type=int, default=300,
                    help='number of hidden units per layer')
parser.add_argument('--nlayers', type=int, default=2,
                    help='number of layers')
parser.add_argument('--lr', type=float, default=20,
                    help='initial learning rate')
parser.add_argument('--clip', type=float, default=0.25,
                    help='gradient clipping')
parser.add_argument('--epochs', type=int, default=150,
                    help='upper epoch limit')
parser.add_argument('--batch_size', type=int, default=48, metavar='N',
                    help='batch size')
parser.add_argument('--bptt', type=int, default=35,
                    help='sequence length')
parser.add_argument('--dropout', type=float, default=0.25,
                    help='dropout applied to layers (0 = no dropout)')
parser.add_argument('--lamda', type=float, default=0.5,
                    help='loss parameter')
parser.add_argument('--tied', action='store_true', default=True,
                    help='tie the word embedding and softmax weights')
parser.add_argument('--seed', type=int, default=20180330,
                    help='random seed')
parser.add_argument('--cuda', action='store_true',
                    help='use CUDA', default=True)
parser.add_argument('--log-interval', type=int, default=200, metavar='N',
                    help='report interval')
parser.add_argument('--save', type=str,  default='/content/gdrive/My Drive/Colab Notebooks/MSProject/SavedModel/model_2.pt',
                    help='path to save the final model')
parser.add_argument('--vocab', type=str, default='/content/gdrive/My Drive/Colab Notebooks/MSProject/Output/VOCAB.txt',
                    help=('preprocessed vocaburary'))
parser.add_argument('--glove', action='store_true', default=True,
                    help='use glove')
parser.add_argument('--glove_path', type=str, default='/content/gdrive/My Drive/Colab Notebooks/MSProject/glove.42B.300d.txt',
                    help='using glove word embedding')
parser.add_argument('--anneal', type=float, default=4,
                    help='anneal rate of learning rate')
#args = parser.parse_args()
##args = parser.parse_args(args=[])
#ap.parse_known_args()[0]
args = parser.parse_args(args=[])
#args, unknown = parser.parse_known_args()
print(args)
# Set the random seed manually for reproducibility.
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")
    # else:
    #     torch.cuda.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()

    return data.to(device)

# 改动1
def repackage_hidden(h):
    """Wraps hidden states in new Tensors, to detach them from their history."""
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

def get_batch(source, i, evaluation=False):
    seq_len = min(args.bptt, len(source) - 1 - i)
    data = Variable(source[i:i+seq_len], volatile=evaluation)
    target = Variable(source[i+1:i+1+seq_len].view(-1))
    return data, target


#改动2
def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, args.bptt):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, ntokens)
            total_loss += len(data) * criterion(output, targets, lamda, f_onehot, m_onehot)
            hidden = repackage_hidden(hidden)
    return total_loss / (len(data_source) - 1)


def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0
    start_time = time.time()
    hidden = model.init_hidden(args.batch_size)
    for batch, i in enumerate(range(0, train_data.size(0) - 1, args.bptt)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        hidden = repackage_hidden(hidden)
        model.zero_grad()
        output, hidden = model(data, hidden)
        #loss = criterion(output.view(-1, ntokens), targets)
        loss = criterion(output, targets, lamda, f_onehot, m_onehot)
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
        ## this is deprecated, no need any more, just use torch.nn.utils.clip_grad_norm
        for p in model.parameters():
            if p.grad is not None:
                p.data.add_(-lr, p.grad.data)
        
        total_loss += loss.data

        if batch % args.log_interval == 0 and batch > 0:
            cur_loss = total_loss.item() / args.log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // args.bptt, lr,
                elapsed * 1000 / args.log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

########################


### LOSS CODE ###

# load doc into memory
def load_doc(filename):
    file = open(filename, 'r', encoding='utf-8-sig')
    text = file.read()
    file.close()
    return text

femaleFile = '/content/gdrive/My Drive/Colab Notebooks/MSProject/Data/WordList/female_word_file.txt'
maleFile = '/content/gdrive/My Drive/Colab Notebooks/MSProject/Data/WordList/male_word_file.txt'

def getGenderIdx(femaleFile, maleFile, word2idx):
    female_word_list = load_doc(femaleFile).split('\n')
    male_word_list = load_doc(maleFile).split('\n')
    pairs = [ (word2idx[f],word2idx[m]) for f,m in zip(female_word_list,male_word_list)  if f in word2idx and m in word2idx]
    femaleIdx = [ f for f,m in pairs]
    maleIdx = [ m for f,m in pairs]
    return femaleIdx,maleIdx

class Custom_Loss(torch.nn.Module):
    
    def __init__(self):
        super(Custom_Loss,self).__init__()
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, output, targets, lamda, femaleIdx, maleIdx):
        cross_ent_loss = self.criterion(output.view(-1, ntokens), targets)
        bias_loss = self.logRatioLossFast(output, femaleIdx, maleIdx) * lamda
        loss = cross_ent_loss + bias_loss     
        return loss
    
    def customLoss_LogRatioGenderPairs(self,output, femaleIdx, maleIdx):
        flato = output.view(-1, ntokens)
        logratio_sum = 0
        for i in range(flato.size()[0]):
            o = flato[i]
            logratio = [abs(torch.log( (torch.exp(o[f]) + 0.00001) / (torch.exp(o[m])+ 0.00001) )) for f,m in zip(femaleIdx, maleIdx)]
            logratio_sum += sum(logratio)    
        return logratio_sum / flato.size()[0]
    
    def logRatioLossFast(self, output, f_onehot, m_onehot):
        flato = output.view(-1, ntokens)
        m1 = torch.matmul(f_onehot,flato.t())
        m2 = torch.matmul(m_onehot,flato.t())
        b_loss = torch.mean(torch.abs(torch.log((torch.exp(m1) + 0.00001) / (torch.exp(m2) + 0.00001))))
        return b_loss


################


#######################

#load vocab            
vocab = preprocess.read_vocab(os.path.join(args.vocab))
#create json file with indexed filename for following separation
inds = [os.path.join(args.data, x) for x in os.listdir(args.data) if x.endswith('bin')]
# make new JSON file
#with open('/content/drive/My Drive/Sandbox/Sandbox_2.json', 'w') as f:
  #f.write('{"hello":true}')

index_train = {}
index_train['id'] = {}
iteration = 0
for ind in inds:
    index_train['id'][iteration] = os.path.basename(ind)
    iteration += 1
with open('ind_train.json', 'w') as fp:
    json.dump(index_train, fp)

    
#load the json file of indexed filename
with open('ind_train.json', 'r') as fp:
    data = json.load(fp)
idx_train_ = pd.DataFrame(data)

print(idx_train_[:10])
#split test set
splitter_tt = ShuffleSplit(n_splits=1, test_size=0.1,
                               random_state=args.seed)
bigtrains, tests = next(splitter_tt.split(idx_train_['id'].keys()))

idx_bigtrain = idx_train_.iloc[bigtrains]
idx_test = idx_train_.iloc[tests]


#split train, val sets
splitter_tv = ShuffleSplit(n_splits=1, test_size=0.2,
                               random_state=args.seed)

trains, vals = next(splitter_tv.split(idx_bigtrain['id'].keys()))

idx_train = idx_bigtrain.iloc[trains]
idx_val = idx_bigtrain.iloc[vals]


#save idx_train, idx_val, idx_test for later use
idx_train.to_json('idx_train.json')
idx_val.to_json('idx_val.json')
idx_test.to_json('idx_test.json')


# Load pretrained Embeddings, common token of vocab and gn_glove will be loaded, only appear in vocab will be initialized
#142527 tokens, last one is '<unk>'
# ntokens = sum(1 for line in open(gn_glove_dir)) + 1
vocab.append('<eos>')
ntokens = len(vocab)
#

with open(args.glove_path,'r+', encoding="utf-8") as f: 
    gn_glove_vecs = np.zeros((142527, 300)) #april_1
    words2idx_emb = {}
    idx2words_emb = []
    # ordered_words = []
    for i, line in enumerate(f):
        try:
            s = line.split() 
            gn_glove_vecs[i, :] = np.asarray(s[1:])
            words2idx_emb[s[0]] = i
            idx2words_emb.append(s[0])
            # ordered_words.append(s[0])
        except:
            continue

    words2idx_emb['<eos>'] = i+1
    idx2words_emb.append('<eos>')

#creat new word embeding, word embedding both  in the gn_glove and vocab keeps, only in vocab is initialized
nw = np.zeros((ntokens, args.emsize), dtype=np.float32)

for i,w in enumerate(vocab):#change add start=1
    try:
        r = words2idx_emb[w]
        nw[i] = gn_glove_vecs[r] 
        test_i += 1 
    except:
        nw[i] = np.random.normal(scale=0.6, size=(args.emsize, ))

words2idx = {item : index for index, item in enumerate(vocab)}
# Load data
corpus = data_v3.Corpus(args.data, vocab, words2idx, idx_train, idx_val, idx_test) #改动2 
eval_batch_size = 10
train_data = batchify(corpus.train, args.batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

# nw = torch.from_numpy(nw)

######################################

femaleIdx,maleIdx = getGenderIdx(femaleFile, maleFile, corpus.words2idx)

f_onehot = np.zeros((len(femaleIdx), ntokens))
f_onehot[np.arange(len(femaleIdx)), femaleIdx] = 1
f_onehot = torch.tensor(f_onehot, dtype = torch.float).to(device)

m_onehot = np.zeros((len(femaleIdx), ntokens))
m_onehot[np.arange(len(femaleIdx)), maleIdx] = 1
m_onehot = torch.tensor(m_onehot, dtype = torch.float).to(device)



###########################################


# Build the model
model = model.RNNModel(args.model, nw, ntokens, args.emsize, args.nhid, args.nlayers, args.dropout, args.tied).to(device)

#criterion = nn.CrossEntropyLoss()
criterion = Custom_Loss().to(device)

# Loop over epochs.
lr = args.lr
best_val_loss = None
lamda = args.lamda

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, args.epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.

        if not best_val_loss or val_loss < best_val_loss:
            with open(args.save, 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            if lr<1e-1 and lr>=1e-2:
                lr /= (args.anneal/2)
            elif lr<1e-2 and lr >=1e-3:
                lr /= (args.anneal/3)
            # elif lr<1e-4 and lr >=1e-5:
            #     lr /= (args.anneal/3.5)
            elif lr<1e-3:
                lr *= 0.95
            else:
                lr /= args.anneal

            print('new learning rate is {}'.format(lr))
            print('val_loss is {}'.format(val_loss))
            print('best_val_loss is {}'.format(best_val_loss))
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

# Load the best saved model.
with open(args.save, 'rb') as f:
    model = torch.load(f)

# Run on test data.
test_loss = evaluate(test_data)
print('=' * 89)
print('| End of training | test loss {:5.3f} | test ppl {:8.3f}'.format(
    test_loss, math.exp(test_loss)))
print('=' * 89)

Namespace(anneal=4, batch_size=48, bptt=35, clip=0.25, cuda=True, data='/content/gdrive/My Drive/Colab Notebooks/MSProject/Output/data', dropout=0.25, emsize=300, epochs=150, glove=True, glove_path='/content/gdrive/My Drive/Colab Notebooks/MSProject/glove.42B.300d.txt', lamda=0.5, log_interval=200, lr=20, model='LSTM', nhid=300, nlayers=2, save='/content/gdrive/My Drive/Colab Notebooks/MSProject/SavedModel/model_2.pt', seed=20180330, tied=True, vocab='/content/gdrive/My Drive/Colab Notebooks/MSProject/Output/VOCAB.txt')
                                                id
0     70630a9ce4576f9b93e759edfab6e90ffa6b16d1.bin
1     20a49e016a4f3807fe24c3caee9af6bd9dc52b5b.bin
10    08ecfda343d2405f633c0331eabc472f3dac4a26.bin
100   8d3cb6429a30a810a8d281ee6e4b8bfc0c928446.bin
1000  31171fd8e19677f021b50a335e490e3fb4680371.bin
1001  0573daee6eff6931eb4ff49a599e80c74c20672e.bin
1002  acbfc239711db1684963b4fe1ddd932e17763067.bin
1003  8143b28f87a6e16d2b59aaf7725b6f4f3102b50f.bin
1004  3a25f0552

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:148: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:152: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)


| epoch   1 |   200/  714 batches | lr 20.00 | ms/batch 82.21 | loss  7.98 | ppl  2915.05
| epoch   1 |   400/  714 batches | lr 20.00 | ms/batch 82.71 | loss  7.02 | ppl  1118.20
| epoch   1 |   600/  714 batches | lr 20.00 | ms/batch 84.62 | loss  6.64 | ppl   766.72
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 67.41s | valid loss  6.45 | valid ppl   634.84
-----------------------------------------------------------------------------------------
| epoch   2 |   200/  714 batches | lr 20.00 | ms/batch 88.20 | loss  6.33 | ppl   563.11
| epoch   2 |   400/  714 batches | lr 20.00 | ms/batch 89.07 | loss  6.17 | ppl   478.20
| epoch   2 |   600/  714 batches | lr 20.00 | ms/batch 88.19 | loss  6.07 | ppl   432.59
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 70.83s | valid loss  6.07 | valid ppl   431.54
------------------------------------------

In [15]:

'''We use a pre-trained 300-dimensional word embedding, GloVe, by Pennington et al. (2014). We
apply random search to the hyperparameter tuning
of the LSTM language model. The best hyperparameters are as follows: 2 hidden layers each with
300 units, a sequence length of 35, a learning rate
of 20 with an annealing schedule of decay starting from 0.25 to 0.95, a dropout rate of 0.25 and
a gradient clip of 0.25. We train our models for
150 epochs, use a batch size of 48, and set early
stopping with a patience of 5'''

'We use a pre-trained 300-dimensional word embedding, GloVe, by Pennington et al. (2014). We\napply random search to the hyperparameter tuning\nof the LSTM language model. The best hyperparameters are as follows: 2 hidden layers each with\n300 units, a sequence length of 35, a learning rate\nof 20 with an annealing schedule of decay starting from 0.25 to 0.95, a dropout rate of 0.25 and\na gradient clip of 0.25. We train our models for\n150 epochs, use a batch size of 48, and set early\nstopping with a patience of 5'